# 第4章：分离数据和指令

- [课程](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt}
        ],
        system=system
    )
    return message.content[0].text

---

## 课程

通常，我们不想写完整的提示，而是希望**提示模板可以在提交给Claude之前用额外的输入数据进行修改**。如果您希望Claude每次都做同样的事情，但Claude用于任务的数据每次可能不同，这可能会很有用。

幸运的是，我们可以通过**将提示的固定骨架与可变用户输入分离，然后在发送完整提示给Claude之前将用户输入替换到提示中**来相当容易地做到这一点。

下面，我们将逐步介绍如何编写可替换的提示模板，以及如何替换用户输入。

### 示例

在第一个示例中，我们让Claude充当动物叫声生成器。请注意，提交给Claude的完整提示只是将输入（在此示例中为"Cow"）替换到`PROMPT_TEMPLATE`中的结果。注意当我们打印完整提示时，单词"Cow"通过f-string替换了占位符`ANIMAL`。

**注意：** 实际使用中，占位变量的名字不必固定。在本示例里我们使用了`ANIMAL`，也可以叫作`CREATURE`或`A`（不过通常让变量名具体且相关更好，这样即使在未替换前阅读提示模板，用户也能理解）。只要确保你在f-string里使用的变量名与模板中的占位符一致即可。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

为什么要这样分离并替换输入？因为**提示模板能简化重复性任务**。设想你构建了一个提示结构，邀请第三方用户向提示中提交内容（本例是他们想生成叫声的动物）。这些用户无需编写甚至无需看到完整提示，只需要填写变量即可。

这里我们用变量和f-string完成替换，你也可以使用`format()`方法。

**注意：** 提示模板可以包含任意多个变量！

当引入这种替换变量时，务必要**明确告诉Claude变量的起止位置**（与指令或任务描述区分开来）。我们来看一个在指令与替换变量之间没有分隔的例子。

对人类读者而言，下面的提示模板里变量的起止非常清晰；但在变量被替换后的完整提示中，这种边界就变得不清楚了。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

在这里，**Claude误以为“Yo Claude”是要重写的邮件内容的一部分**！你可以从它以“Dear Claude”开头的重写结果看出来。对人类而言（尤其在提示模板里）邮件的起止很清楚，但在变量替换后的完整提示中，这一点就不那么清楚了。

该如何解决？**用XML标签包裹输入**！如下所示，这样做后输出中不再出现“Dear Claude”。

[XML 标签](https://docs.anthropic.com/claude/docs/use-xml-tags) 是尖括号形式的标签，如 `<tag></tag>`。它们成对出现，由一个起始标签（如 `<tag>`）和一个以`/`标记的闭合标签（如 `</tag>`）组成。XML 标签用于包裹内容，例如：`<tag>content</tag>`。

**注意：** 虽然Claude可以识别并处理多种分隔符，我们仍然建议**优先使用XML标签作为分隔符**，因为Claude在训练中被特别强化用于把XML标签作为提示的组织机制。除函数调用场景外，**并不存在什么“特效”的XML标签能显著提升表现**。我们有意让Claude在这方面保持高度可塑性与可定制性。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

让我们看另一个XML标签如何帮助我们的例子。

在下面的提示中，**Claude错误地解释了提示的哪一部分是指令与输入**。由于格式的原因，它错误地认为`Each is about an animal, like rabbits`是列表的一部分，而用户（填写`SENTENCES`变量的人）显然不希望如此。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

要修复这个问题，我们只需要**用XML标签包围用户输入的句子**。这样就能告诉Claude输入数据的起止位置，避免被`Each is about an animal, like rabbits.`前面的连字符误导。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

**注意：** 在"Each is about an animal"提示的错误版本中，我们必须包含连字符才能让Claude按照我们示例所需的方式错误响应。这是关于提示的重要教训：**细节很重要**！值得**仔细检查提示中的拼写和语法错误**。Claude对模式很敏感（在早期，在微调之前，它是一个原始的文本预测工具），当你犯错误时它更容易犯错误，当你听起来聪明时它更聪明，当你听起来愚蠢时它更愚蠢，等等。

如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 4.1 - 俳句主题](#exercise-41---haiku-topic)
- [练习 4.2 - 带拼写错误的狗问题](#exercise-42---dog-question-with-typos)
- [练习 4.3 - 狗问题第2部分](#exercise-42---dog-question-part-2)

### 练习 4.1 - 俳句主题
修改`PROMPT`使其成为一个模板，接受名为`TOPIC`的变量并输出关于该主题的俳句。这个练习只是为了测试您对f-string变量模板结构的理解。

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f""

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
print(hints.exercise_4_1_hint)

### 练习 4.2 - 带拼写错误的狗问题
通过添加XML标签来修复`PROMPT`，使Claude产生正确答案。

尽量不要更改提示的其他任何内容。混乱和错误百出的写法是故意的，这样您可以看到Claude如何对这些错误做出反应。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
print(hints.exercise_4_2_hint)

### 练习 4.3 - 狗问题第2部分
**不使用**XML标签来修复`PROMPT`。相反，只从提示中删除一到两个单词。

与上面的练习一样，尽量不要更改提示的其他任何内容。这将向您展示Claude可以解析和理解什么样的语言。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
print(hints.exercise_4_3_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，那么您已经准备好进入下一章了。祝您提示愉快！

---

## 示例练习场

这是一个供您自由试验本课中展示的提示示例的区域，您可以调整提示来看看它如何影响Claude的回应。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))